In [1]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [3]:
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the Llama-3.2-3B-Instruct model and tokenizer
model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define houses with descriptions and true values (hidden from agents)
houses = [
    {"name": "House A", "desc": "Small urban apartment", "start_price": 100, "true_value": 300},
    {"name": "House B", "desc": "Large rural farmhouse", "start_price": 150, "true_value": 400},
    {"name": "House C", "desc": "Eco-friendly suburban home", "start_price": 120, "true_value": 350},
]

# Personas and their preferences
personas = {
    "Urban Dweller": ["urban"],
    "Rural Seeker": ["rural"],
    "Eco-Conscious": ["eco-friendly"],
}

# Agent class to handle bidding, profiling, and strategy
class Agent:
    def __init__(self, name, persona=None, budget=1000):
        self.name = name
        self.persona = persona
        self.budget = budget
        self.won_items = []
        self.profit = 0
        self.priority_list = {}
        self.profiles = {}  # Store inferred profiles of competitors

    def generate_response(self, prompt):
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        outputs = model.generate(
            **inputs, max_new_tokens=200, temperature=0.7, do_sample=True
        )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    def initialize_priority(self, houses):
        prompt = (
            f"You are {self.name}. "
            f"{'Your persona is ' + self.persona + '.' if self.persona else 'You have no persona.'} "
            f"Your budget is ${self.budget}. This auction has {len(houses)} items: "
            f"{[h['name'] + ' (' + h['desc'] + ')' for h in houses]}. "
            "Assign a priority (1-3) to each item based on your persona and profit objective. "
            "Output in JSON: {'House A': 3, 'House B': 2, ...}"
        )
        response = self.generate_response(prompt)
        try:
            import json
            self.priority_list = json.loads(response.split("```json")[1].split("```")[0])
        except:
            # Fallback in case JSON parsing fails
            self.priority_list = {h["name"]: random.randint(1, 3) for h in houses}

    def profile_competitors(self, house, bidding_history, competitors):
        prompt = (
            f"You are {self.name}. Bidding history for {house['name']} ({house['desc']}): {bidding_history}. "
            f"Persona list: {list(personas.keys())}. For each competitor, predict their persona weights "
            f"(-1 to 1) based on their bidding. Output in JSON: {{'Bidder': {{'Persona': weight, ...}}}}"
        )
        response = self.generate_response(prompt)
        try:
            import json
            self.profiles = json.loads(response.split("```json")[1].split("```")[0])
        except:
            self.profiles = {comp.name: {p: 0 for p in personas} for comp in competitors}

    def decide_bid(self, house, current_bid, bidding_history, competitors):
        prompt = (
            f"You are {self.name}. Auction for {house['name']} ({house['desc']}). "
            f"Current bid: ${current_bid}. Your budget: ${self.budget}. "
            f"Priority: {self.priority_list.get(house['name'], 1)}. "
            f"Profiles: {self.profiles}. Bidding history: {bidding_history}. "
            "Choose: A) Increase bid if top priority, B) Increase to drain rivals, C) Bid if budget allows, "
            "D) Quit to conserve, E) Quit if profit low, F) Quit to avoid war. Output: 'Action: X, Bid: $Y' or 'Action: Quit'"
        )
        response = self.generate_response(prompt)
        if "Quit" in response:
            return None
        try:
            bid = int(response.split("Bid: $")[1])
            if bid > self.budget or bid <= current_bid:
                return None
            return bid
        except:
            return None

    def update_status(self, house, bid, won):
        if won:
            self.budget -= bid
            self.profit += house["true_value"] - bid
            self.won_items.append(house["name"])

# Auction simulation
def run_auction():
    agents = [
        Agent("Master", None),
        Agent("Rival 1", "Urban Dweller"),
        Agent("Rival 2", "Eco-Conscious"),
    ]

    # Initialize priorities
    for agent in agents:
        agent.initialize_priority(houses)

    # Run auction for each house
    for house in houses:
        print(f"\nAuctioning {house['name']} (Starting Price: ${house['start_price']})")
        current_bid = house["start_price"]
        active_agents = agents.copy()
        bidding_history = []

        while len(active_agents) > 1:
            for agent in active_agents[:]:
                # Profile competitors
                agent.profile_competitors(house, bidding_history, [a for a in agents if a != agent])
                # Decide bid
                bid = agent.decide_bid(house, current_bid, bidding_history, [a for a in agents if a != agent])
                if bid:
                    current_bid = bid
                    bidding_history.append(f"{agent.name} bids ${bid}")
                    print(f"{agent.name} bids ${bid}")
                else:
                    active_agents.remove(agent)
                    bidding_history.append(f"{agent.name} quits")
                    print(f"{agent.name} quits")
            if len(active_agents) == 1:
                winner = active_agents[0]
                winner.update_status(house, current_bid, True)
                print(f"{winner.name} wins {house['name']} for ${current_bid}")
                break
            if not active_agents:
                print(f"No winner for {house['name']}")
                break

    # Display results
    print("\nFinal Results:")
    for agent in agents:
        print(f"{agent.name}: Budget: ${agent.budget}, Profit: ${agent.profit}, Won: {agent.won_items}")

if __name__ == "__main__":
    run_auction()

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

Master quits


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Rival 1 quits


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Rival 2 quits
No winner for House C

Final Results:
Master: Budget: $1000, Profit: $0, Won: []
Rival 1: Budget: $1000, Profit: $0, Won: []
Rival 2: Budget: $1000, Profit: $0, Won: []
